In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib.request as libreq

In [2]:
soup = BeautifulSoup(open(r'classification_codes_data\arxiv_category_taxonomy.html'), 'html.parser')
classification_codes = soup.get_text().split('\n')

In [27]:
main_cls = ['Computer Science', 'Economics', 'Electrical Engineering and Systems Science', 'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']

In [3]:
# classification_codes = [code for code in classification_codes if (code != '' and code not in main_cls)]
classification_codes = [code for code in classification_codes if (code != '')]

In [6]:
classification_codes = pd.DataFrame(classification_codes, columns=['classification_codes'])
# classification_codes.to_csv(r'arxiv_classification.txt', index=False)

In [30]:
df = pd.read_csv(r'arxiv_classification.csv')
df

,class,description
0,cs.AI (Artificial Intelligence),"""Covers all areas of AI except Vision Robot..."
1,cs.AR (Hardware Architecture),"""Covers systems organization and hardware ar..."
2,cs.CC (Computational Complexity),"""Covers models of computation complexity cl..."
3,cs.CE (Computational Engineering Finance and...,"""Covers applications of computer science to ..."
4,cs.CG (Computational Geometry),"""Roughly includes material in ACM Subject Cl..."
...,...,...
150,stat.CO (Computation),"""Algorithms Simulation Visualization"""
151,stat.ME (Methodology),"""Design Surveys Model Selection Multiple ..."
152,stat.ML (Machine Learning),"""Covers machine learning papers (supervised ..."
153,stat.OT (Other Statistics),Work in statistics that does not fit into th...


In [31]:
df['code'] = df['class'].apply(lambda x: x.split(' '))
df['class_title'] = df['code'].apply(lambda x: ' '.join(x[1:]))
df['class_code'] = df['code'].apply(lambda x: x[0])

df['description'] = df['description'].apply(lambda x: x.replace('"',''))

df.drop(['code','class'], axis=1, inplace=True)
df

KeyError: 'description'